In [2]:
! pip install requests

  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Using cached charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
! pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from io import BytesIO

In [7]:
url = "https://ticket.interpark.com/TPGoodsList.asp?Ca=Mus"
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

# 초기화
interpark_df = pd.DataFrame()

r_list = requests.get(url)

soup = BeautifulSoup(r_list.text, 'html.parser')
tmp_href = soup.find_all("span", class_="fw_bold")

for span_tag in tmp_href:
    a_tag = span_tag.find('a')
    href_value = a_tag['href']

    # goods_code id값 추출
    goods_code = href_value.split('=')[1]
    img_code = goods_code[:2]
    
    # api 추출
    info_url = f"https://api-ticketfront.interpark.com/v1/goods/{goods_code}/summary?goodsCode={goods_code}&priceGrade=&seatGrade="
    story_url = f"https://api-ticketfront.interpark.com/v1/goods/play-db?goodsCode={goods_code}"
    img_url = f"https://ticketimage.interpark.com/Play/image/large/{img_code}/{goods_code}_p.gif"
    
    # # 이미지 저장
    # r_img = requests.get(img_url)
    # if r_img.status_code == 200:
    #     image_data = r_img.content
    #     image = Image.open(BytesIO(image_data))
    #     image.save(f"main_img/{goods_code}.png") 
    # else:
    #     print("Failed to retrieve data from the URL")
    
    # 공연정보 df 변환
    r_info = requests.get(info_url, headers=header)
    if r_info.status_code == 200:
        json_data = r_info.json()
    else:
        print("Failed to retrieve data from the URL")
        
    data = json_data['data']
    df = pd.DataFrame([data])
    
    interpark_df = pd.concat([interpark_df, df], ignore_index=True)

interpark_df

,goodsCode,goodsName,subGoodsName,goodsNameEucKr,placeCode,placeName,genreCode,genreName,genreSubCode,genreSubName,...,isUsedCastingCalendar,maxSalesPrice,minSalesPrice,mediaUrl,priceGradeName,mobileInfo,sessionStartDate,sessionEndDate,soldOut,isGoodsTicketCast
0,23008707,태양의서커스 〈루치아〉,,%C5%C2%BE%E7%C0%C7%BC%AD%C4%BF%BD%BA+%A1%B4%B7...,23000678,잠실종합운동장 내 빅탑,01011,뮤지컬,11007,오리지널/내한공연,...,False,0,0,http://ticketimage.interpark.com/PlayDictionar...,None,None,None,None,None,False
1,23014423,뮤지컬 〈몬테크리스토〉,,%B9%C2%C1%F6%C4%C3+%A1%B4%B8%F3%C5%D7%C5%A9%B8...,22000525,충무아트센터 대극장,01011,뮤지컬,11008,라이선스,...,True,0,0,,None,None,None,None,None,False
2,23012526,뮤지컬 〈레미제라블〉,,%B9%C2%C1%F6%C4%C3+%A1%B4%B7%B9%B9%CC%C1%A6%B6...,22000001,블루스퀘어 신한카드홀,01011,뮤지컬,11008,라이선스,...,True,0,0,http://ticketimage.interpark.com/PlayDictionar...,None,None,None,None,None,False
3,23013512,뮤지컬 드라큘라 (Dracula：The Musical),,%B9%C2%C1%F6%C4%C3+%B5%E5%B6%F3%C5%A7%B6%F3+%2...,22000338,샤롯데씨어터,01011,뮤지컬,11008,라이선스,...,True,0,0,,None,None,None,None,None,False
4,23013171,뮤지컬 〈렌트〉,,%B9%C2%C1%F6%C4%C3+%A1%B4%B7%BB%C6%AE%A1%B5,23001252,coex 신한카드 artium,01011,뮤지컬,11008,라이선스,...,True,0,0,http://ticketimage.interpark.com/PlayDictionar...,None,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,23015408,MUHI,,MUHI,21001212,드림시어터 소극장,01011,뮤지컬,11009,창작뮤지컬,...,False,0,0,,None,None,None,None,None,False
160,23016219,My name is 대불 - 구리,대단히 불같은 그녀,My+name+is+%B4%EB%BA%D2+-+%B1%B8%B8%AE,12011463,구리아트홀 유채꽃 소극장,01011,뮤지컬,11009,창작뮤지컬,...,False,0,0,,None,None,None,None,None,False
161,23015019,NEW 달을 품은 슈퍼맨 - 대전,,NEW+%B4%DE%C0%BB+%C7%B0%C0%BA+%BD%B4%C6%DB%B8%...,18000237,대전 상상아트홀,01011,뮤지컬,11009,창작뮤지컬,...,False,0,0,,None,None,None,None,None,False
162,23015655,NOL 스테이지 〈뮤지컬 레미제라블〉 미니콘서트,,NOL+%BD%BA%C5%D7%C0%CC%C1%F6+%A1%B4%B9%C2%C1%F...,23001499,현대카드 UNDERSTAGE,01011,뮤지컬,11008,라이선스,...,False,0,0,,None,None,None,None,None,False


In [8]:
interpark_df.to_csv("interpark_info.csv", index=False, encoding="cp949")

In [9]:
interpark_df.to_csv("interpark_info2.csv", index=False, encoding="utf-8")


In [10]:
info_url

'https://api-ticketfront.interpark.com/v1/goods/23011767/summary?goodsCode=23011767&priceGrade=&seatGrade='